In [1]:
from bigml.api import BigML
api = BigML(project='project/5d94a426eba31d46690001cd')

We retrieve the stored python objetcs.

In [2]:
%store -r train_training_dataset
%store -r train_validation_dataset

We create an Ensemble model from the *train_training_dataset*.

In [3]:
deepnet = api.create_deepnet(train_training_dataset, {"objective_field": "SeriousDlqin2yrs"})

We create a batch prediction from this Ensemble onto the *train_validation_dataset*, keeping all fields + **Prediction, Probability, 0 probability** and **1 probability**.

In [4]:
batch_prediction = api.create_batch_prediction(deepnet, train_validation_dataset,
                                               {"all_fields": True, "header": True, "prediction_name": "Prediction", "probability": True, "probabilities": True})

api.ok(batch_prediction)

api.download_batch_prediction(batch_prediction,
                              filename='./deepnet_train_prediction.csv')

'./deepnet_train_prediction.csv'

We create an evaluation from the previous model with the same settings.

In [7]:
evaluation_deepnet = api.create_evaluation(deepnet, train_validation_dataset, {"name": "Eval ensemble", "operating_point": {"threshold": 0.5, "positive_class": "1"}})
api.ok(evaluation_deepnet)
%store evaluation_deepnet

Stored 'evaluation_deepnet' (dict)


We create a new dataframe with pandas from the previous evaluation object. **The positive class will be 0 though** (can't change it atm for some reason...).

In [9]:
import pandas
eval = pandas.DataFrame(evaluation_deepnet["object"]["result"]["model"]["confusion_matrix"], index=["Predicted F", "Predicted P"], columns=["Actual F", "Actual P"])
print(eval)

             Actual F  Actual P
Predicted F     27255       688
Predicted P      1423       634
